In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import ast

In [3]:
total = pd.DataFrame()

In [4]:
iterations = ['']
for i in range(101,5701,100):
    iterations.append('/' + str(i))

In [5]:
pageCount = 0
for iteration in iterations:
    print('Starting crawl for the ' + str(pageCount) + 'th page')
    url = 'https://www.the-numbers.com/movie/budgets/all' + iteration
    print(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    # get title
    aref = rows[1].find_all('a')
    titles = []
    for index, value in enumerate(aref):
        if index % 2 == 1:
            titles.append(value.text)
    # get numeric data
    data = rows[1].find_all('td', class_='data')
    budget = []
    worldGross = []
    for index, value in enumerate(data):
        if index % 4 == 1:
            budget.append(value.text)
        elif index % 4 == 3:
            worldGross.append(value.text)
    temp = pd.DataFrame({'title': titles, 'budget':budget, 'gross':worldGross})
    total = total.append(temp)
    pageCount += 1

Starting crawl for the 0th page
https://www.the-numbers.com/movie/budgets/all
Starting crawl for the 1th page
https://www.the-numbers.com/movie/budgets/all/101
Starting crawl for the 2th page
https://www.the-numbers.com/movie/budgets/all/201
Starting crawl for the 3th page
https://www.the-numbers.com/movie/budgets/all/301
Starting crawl for the 4th page
https://www.the-numbers.com/movie/budgets/all/401
Starting crawl for the 5th page
https://www.the-numbers.com/movie/budgets/all/501
Starting crawl for the 6th page
https://www.the-numbers.com/movie/budgets/all/601
Starting crawl for the 7th page
https://www.the-numbers.com/movie/budgets/all/701
Starting crawl for the 8th page
https://www.the-numbers.com/movie/budgets/all/801
Starting crawl for the 9th page
https://www.the-numbers.com/movie/budgets/all/901
Starting crawl for the 10th page
https://www.the-numbers.com/movie/budgets/all/1001
Starting crawl for the 11th page
https://www.the-numbers.com/movie/budgets/all/1101
Starting crawl f

In [12]:
total

,title,budget,gross
0,Avatar,"$425,000,000","$2,776,345,279"
1,Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$1,045,663,875"
2,Avengers: Age of Ultron,"$330,600,000","$1,405,057,855"
3,Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$1,316,764,784"
4,Star Wars Ep. VII: The Force Awakens,"$306,000,000","$2,053,311,220"
5,Avengers: Infinity War,"$300,000,000","$2,008,425,124"
6,Pirates of the Caribbean: At Worldâs End,"$300,000,000","$963,420,425"
7,Justice League,"$300,000,000","$655,953,446"
8,Spectre,"$300,000,000","$879,620,923"
9,The Dark Knight Rises,"$275,000,000","$1,084,439,099"


In [15]:
def getData():
    print('Reading meta data csv.')
    mainDS = pd.read_csv("../Documents/movies-profit/movies_metadata.csv")
    mainDS = mainDS.drop([19730,29503,35587])
    mainDS.drop(['adult','homepage','original_language','overview','spoken_languages','status','tagline','original_title','video', 'poster_path'], axis=1, inplace = True)
    return mainDS

def transformCollection(mainDS):
    # Convert if belongs to a collection to a boolean value.
    print('Transforming the collections column.')
    collections = mainDS['belongs_to_collection']
    isCollection = ~collections.isnull()
    mainDS['belongs_to_collection'] = isCollection
    return mainDS

def getUniqueAndCleanGenres(mainDS):
    print('Cleaning Genres and getting unique list.')
    genres = mainDS['genres'].tolist()
    genreList = []
    cleanGenres = []
    for rawGenre in genres:
        cleanGenre = ast.literal_eval(rawGenre)
        genreString = ''
        for element in cleanGenre:
            genreList.append(element['name'])
            genreString = genreString + element['name'] + ','
        cleanGenres.append(genreString.strip(','))
    mainDS['genres'] = cleanGenres 
    # Set gets unique values and then convert back into list
    uniqueGenres = list(set(genreList))
    return uniqueGenres, mainDS

def castIdToInt(mainDS):
    print('Converting ID to integer data type.')
    ids = mainDS['id'].values
    idList = []
    index = 0
    for item in ids:
        try:
            idList.append(int(item))
            index += 1
        except:
            print(index)
            index += 1
    mainDS['id'] = idList
    return mainDS

def getCastData(mainDS):
    print('Reading cast data. Merging with main dataset.')
    creditData = pd.read_csv('../Documents/movies-profit/act_dirDS.csv')
    mergedDS = mainDS.merge(creditData, how = 'inner', left_on = 'id', right_on = 'id')

    directors = mergedDS['director'].tolist()
    cleanDirectors = []
    for directorString in directors:
        director = directorString.strip('[]')
        director = director.replace("'", "")
        cleanDirectors.append(director)
    mergedDS['director'] = cleanDirectors

    actors = mergedDS['actors'].tolist()
    cleanActors = []
    for actorString in actors:
        actor = actorString.strip('[]')
        actor = actor.replace("'", "")
        cleanActors.append(actor)
    mergedDS['actors'] = cleanActors
    return mergedDS

def removeNulls(mainDS):
    print('Removing nulls and 0 values.')
    mainDS.dropna(axis = 0, how = 'any', inplace = True)
    mainDS = mainDS.reset_index(drop = True)
    return mainDS

In [16]:
mainDS = getData()
mainDS = transformCollection(mainDS)
uniqueGenres, mainDS = getUniqueAndCleanGenres(mainDS)
mainDS = castIdToInt(mainDS)
mainDS = getCastData(mainDS)
mainDS = removeNulls(mainDS)
mainDS = mainDS[mainDS['budget']!=0]
mainDS = mainDS.dropna(axis = 0, how = 'any')
print('Done')

Reading meta data csv.
Transforming the collections column.
Cleaning Genres and getting unique list.
Converting ID to integer data type.
Reading cast data. Merging with main dataset.
Removing nulls and 0 values.
Done


In [17]:
data.shape

AttributeError: ResultSet object has no attribute 'shape'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [18]:
mask1 = merged['budget'] == 0
mask2 = merged['revenue'] == 0
data = merged[mask1 & mask2]

NameError: name 'merged' is not defined

In [216]:
data = data.merge(total, 'inner', left_on = 'title', right_on = 'title')
data['budget_x'] = data['budget_y']
data['revenue'] = data['gross']
data = data.rename({'budget_x': 'budget'}, axis = 1)
data = data.drop(['budget_y', 'gross'], axis = 1)

In [217]:
data

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,director,actors
0,False,False,"$20,000,000","Drama,Thriller",1710,tt0112722,en,Copycat,10.701801,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-27,"$32,051,917",124.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Copycat,6.5,199.0,['Jon Amiel'],"['Sigourney Weaver', 'Holly Hunter', 'Will Pat..."
1,False,False,"$11,000,000",Drama,16420,tt0114057,en,Othello,1.845899,"[{'name': 'Columbia Pictures', 'id': 5}, {'nam...","[{'iso_3166_1': 'IT', 'name': 'Italy'}]",1995-12-15,"$2,844,379",123.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Othello,7.0,33.0,['Oliver Parker'],"['Laurence Fishburne', 'Irène Jacob', 'Kenneth..."
2,False,False,"$11,000,000","Drama,Romance",47697,tt0045251,en,Othello,2.698585,"[{'name': 'Les Films Marceau', 'id': 2260}, {'...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1951-11-27,"$2,844,379",90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Othello,6.5,35.0,['Orson Welles'],"['Orson Welles', 'Micheál MacLiammóir', 'Rober..."
3,False,False,"$11,000,000",Drama,44006,tt0059555,en,Othello,0.377325,"[{'name': 'BHE Films', 'id': 2008}, {'name': '...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1965-12-15,"$2,844,379",165.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Othello,6.1,9.0,['Stuart Burge'],"['Laurence Olivier', 'Maggie Smith', 'Joyce Re..."
4,False,False,"$11,000,000",NaN,119928,tt0082861,en,Othello,0.653579,[],[],1981-10-04,"$2,844,379",204.0,[],Othello,9.5,2.0,['Colin Lowrey'],['Anthony Hopkins']
5,False,False,"$9,200,000","Drama,War",31174,tt0114279,en,Richard III,4.561387,"[{'name': 'British Screen', 'id': 871}, {'name...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...",1995-12-29,"$4,199,334",104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Richard III,6.9,50.0,['Richard Loncraine'],"['Ian McKellen', 'Annette Bening', 'Jim Broadb..."
6,False,False,"$9,200,000","Drama,History",43323,tt0049674,en,Richard III,1.971284,"[{'name': 'London Film Productions', 'id': 659...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1955-12-13,"$4,199,334",161.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Richard III,6.8,26.0,['Laurence Olivier'],"['Cedric Hardwicke', 'Nicholas Hannen', 'Laure..."
7,False,False,"$9,200,000",Drama,46758,tt0002461,en,Richard III,0.051310,"[{'name': ""Le Film d'Art"", 'id': 1573}, {'name...","[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso...",1912-10-15,"$4,199,334",55.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Richard III,0.0,1.0,"['André Calmettes', 'James Keane']","['Robert Gemp', 'Frederick Warde', 'Albert Gar..."
8,False,False,"$80,000,000","Comedy,Drama,Romance",11010,tt0110877,it,Il postino,10.168437,[],"[{'iso_3166_1': 'BE', 'name': 'Belgium'}, {'is...",1994-09-22,"$20,841,123",108.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",The Postman,7.6,181.0,['Michael Radford'],"['Philippe Noiret', 'Massimo Troisi', 'Maria G..."
9,False,False,"$65,000,000","Drama,History",48750,tt0113828,fr,Les misérables,1.479446,"[{'name': 'Les Films 13', 'id': 1742}, {'name'...","[{'iso_3166_1': 'FR', 'name': 'France'}]",1995-03-22,"$442,169,052",175.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Les Miserables,6.6,8.0,['Claude Lelouch'],"['Jean-Paul Belmondo', 'Michel Boujenah', 'Ale..."


In [218]:
data.shape

(1562, 20)

In [214]:
merged.shape

(45538, 20)

In [219]:
mask1 = merged['budget'] != 0
mask2 = merged['revenue'] != 0
merged = merged[mask1 & mask2]

In [220]:
merged.shape

(5393, 20)

In [203]:
merged = pd.read_csv('../Documents/movies-profit/mergedData.csv')

In [205]:
merged.shape

(45538, 20)

In [206]:
merged.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,title,vote_average,vote_count,director,actors
0,False,True,30000000,"Animation,Comedy,Family",862,tt0114709,en,Toy Story,21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Toy Story,7.7,5415.0,['John Lasseter'],"['Tom Hanks', 'Tim Allen', 'Don Rickles']"
1,False,False,65000000,"Adventure,Fantasy,Family",8844,tt0113497,en,Jumanji,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Jumanji,6.9,2413.0,['Joe Johnston'],"['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."
2,False,True,0,"Romance,Comedy",15602,tt0113228,en,Grumpier Old Men,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Grumpier Old Men,6.5,92.0,['Howard Deutch'],"['Walter Matthau', 'Jack Lemmon', 'Ann-Margret']"
3,False,False,16000000,"Comedy,Drama,Romance",31357,tt0114885,en,Waiting to Exhale,3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Waiting to Exhale,6.1,34.0,['Forest Whitaker'],"['Whitney Houston', 'Angela Bassett', 'Loretta..."
4,False,True,0,Comedy,11862,tt0113041,en,Father of the Bride Part II,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Father of the Bride Part II,5.7,173.0,['Charles Shyer'],"['Steve Martin', 'Diane Keaton', 'Martin Short']"


In [226]:
new.shape

(6955, 20)

In [238]:
test = new.iloc[6929]['revenue']

In [242]:
int(test.replace('$', ''))

0

In [229]:
new = new.reset_index(drop = True)

In [221]:
new = merged.append(data)

In [222]:
new.shape

(6955, 20)

In [230]:
new.to_csv('newData.csv')